In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
# Load Data
df = pd.read_csv("bangkok_weather.csv")
df['last_updated'] = pd.to_datetime(df['last_updated'], errors='coerce')
df['last_updated'] = df['last_updated'].dt.strftime('%Y-%m-%d')

In [3]:
# Header
header = widgets.HTML("""
    <div style='text-align:center;background-color:#f7f9fc;
    padding:15px;border-radius:10px;margin-bottom:10px;
    box-shadow:0px 2px 4px rgba(0,0,0,0.1);'>
    <h1 style='color:#2c3e50;font-family:Helvetica;'>🌤 Bangkok Weather & Air Quality Dashboard</h1>
    <p style='color:#555;'>Interactive weather, air quality, and correlation insights</p>
    </div>
""")

In [4]:
# Navigation buttons
nav = widgets.ToggleButtons(
    options=["🌤 Weather Overview", "🌫 Air Quality", "📈 Correlations"],
    button_style="info",
    layout=widgets.Layout(width="auto", justify_content="center"),
)

In [5]:
### Filters ###

# Standard filters that appear everypage
year_check = widgets.SelectMultiple(
    options=sorted(df["year"].unique()),
    value=tuple(sorted(df["year"].unique())),
    layout=widgets.Layout(width="100%", height="100px"),
)
month_check = widgets.SelectMultiple(
    options=sorted(df["month"].unique()),
    value=tuple(sorted(df["month"].unique())),
    layout=widgets.Layout(width="100%", height="100px"),
)
day_check = widgets.SelectMultiple(
    options=df["day_of_week"].unique(),
    value=tuple(df["day_of_week"].unique()),
    layout=widgets.Layout(width="100%", height="120px"),
)

# Pollutants filter (Air Quality page)
air_pollution_list = [
    "air_quality_PM2.5", "air_quality_PM10",
    "air_quality_Carbon_Monoxide", "air_quality_Ozone",
    "air_quality_Sulphur_dioxide", "air_quality_Nitrogen_dioxide",
]
pollutant_check = widgets.SelectMultiple(
    options=air_pollution_list,
    value=tuple(air_pollution_list),
    layout=widgets.Layout(width="100%", height="140px"),
)

# Temperature toggle (Overview page)
temp_checkbox = widgets.Checkbox(
    value=False, description="Show °F instead of °C", indent=False,
    layout=widgets.Layout(width="auto", margin="5px 0 0 10px"),
)

# Wind speed toggle (Overview page)
wind_checkbox = widgets.Checkbox(
    value=False, description="Show mph instead of km/h", indent=False,
    layout=widgets.Layout(width="auto", margin="5px 0 0 10px"),
)

In [6]:
# Apply Style to filters
def styled_box(title, content):
    # Apply style condition to the filters
    # Only wind speed need more width
    if title == "Wind Speed Unit":
        return widgets.VBox([
            widgets.HTML(f"<b style='font-size:14px;color:#2c3e50;'>{title}</b>"),
            content
        ], layout=widgets.Layout(
            border='1px solid #d0d7de', padding='10px', border_radius='10px',
            width='220px', background_color='#fdfdfd'
        ))
    else:
        return widgets.VBox([
            widgets.HTML(f"<b style='font-size:14px;color:#2c3e50;'>{title}</b>"),
            content
        ], layout=widgets.Layout(
            border='1px solid #d0d7de', padding='10px', border_radius='10px',
            width='200px', background_color='#fdfdfd'
        ))
    


year_box = styled_box("Year", year_check)
month_box = styled_box("Month", month_check)
day_box = styled_box("Day of Week", day_check)
pollutant_box = styled_box("Pollutants", pollutant_check)
temp_check_box = styled_box("Temperature Unit", temp_checkbox)
wind_check_box = styled_box("Wind Speed Unit", wind_checkbox)

In [7]:
# OVERVIEW PAGE
def weather_graphs(df_sub, use_f=False, use_mph=False):
    figs = []

    # --- Temperature Plot ---
    temps = df_sub["temperature_celsius"]
    label = "°C"
    if use_f:
        temps = df_sub["temperature_fahrenheit"]
        label = "°F"

    fig1 = px.line(
        df_sub,
        x="last_updated",
        y=temps,
        title=f"Temperature ({label})",
        labels={"x": "Date", "y": f"Temperature ({label})"},
        width=475, 
        height=400
    )
    fig1.update_traces(line=dict(color="#f39c12"))
    figs.append(fig1)

    # --- Humidity Plot ---
    fig2 = px.line(
        df_sub,
        x="last_updated",
        y="humidity",
        title="Humidity (%)",
        labels={"x": "Date", "y": "Humidity (%)"},
        width=475,
        height=400
    )
    fig2.update_traces(line=dict(color="#16a085"))
    figs.append(fig2)

    # --- Wind Speed Plot ---
    wind_label = "km/h"
    wind_col = "wind_kph"
    if use_mph:
        wind_col = "wind_mph"
        wind_label = "mph"

    fig3 = px.line(
        df_sub,
        x="last_updated",
        y=wind_col,
        title=f"Wind Speed ({wind_label})",
        labels={"x": "Date", "y": f"Wind Speed ({wind_label})"},
        width=475, 
        height=400
    )
    fig3.update_traces(line=dict(color="#2980b9"))
    figs.append(fig3)

    # --- Rainfall Plot (Average Monthly) ---
    monthly = df_sub.groupby("month")["precip_mm"].mean().dropna().reset_index()
    fig4 = px.bar(
        monthly,
        x="month",
        y="precip_mm",
        title="Avg Monthly Rainfall (mm)",
        labels={"month": "Month", "precip_mm": "Rainfall (mm)"},
        width=450,
        height=400
    )
    fig4.update_traces(marker_color="#3498db")
    figs.append(fig4)

    return figs

In [8]:
# AIR QUALITY PAGE
def air_graphs(df_sub, pollution):
    figs = []

    # --- All pollutants line graph ---
    fig1 = go.Figure()
    for p in pollution:
        if p in df_sub.columns:
            fig1.add_trace(go.Scatter(
                x=df_sub["last_updated"],
                y=df_sub[p],
                mode="lines",
                name=p
            ))
    fig1.update_layout(
        title="Air Quality Pollutants",
        xaxis_title="Date",
        yaxis_title="Pollutant Levels",
        legend=dict(font=dict(size=10))
    )
    figs.append(fig1)

    # --- US EPA AQI Histogram ---
    fig2 = px.histogram(
        df_sub,
        x="air_quality_us-epa-index",
        nbins=10,
        title="US EPA AQI Distribution",
        labels={"air_quality_us-epa-index": "US EPA AQI"}
    )
    fig2.update_traces(marker_color="#e74c3c")
    figs.append(fig2)

    return figs

In [9]:
# CORRELATIONS PAGE
def corr_graphs(df_sub):
    figs = []

    # --- Temperature vs Humidity ---
    if "temperature_celsius" in df_sub.columns and "humidity" in df_sub.columns:
        fig1 = px.scatter(
            df_sub,
            x="temperature_celsius",
            y="humidity",
            title="Temperature vs Humidity",
            labels={"temperature_celsius": "Temperature (°C)", "humidity": "Humidity (%)"},
            opacity=0.6
        )
        fig1.update_traces(marker=dict(color="#27ae60"))
        figs.append(fig1)

    # --- Cloud % vs UV Index ---
    if "cloud" in df_sub.columns and "uv_index" in df_sub.columns:
        fig2 = px.scatter(
            df_sub,
            x="cloud",
            y="uv_index",
            title="Cloud % vs UV Index",
            labels={"cloud": "Cloud (%)", "uv_index": "UV Index"},
            opacity=0.6
        )
        fig2.update_traces(marker=dict(color="#8e44ad"))
        figs.append(fig2)

    return figs

In [10]:
# Render Page + Output Graphs + Filters
output = widgets.Output()

def render_page(df_sub, page):
    # Toggle slicer visibility based on selected page
    pollutant_box.layout.display = "flex" if page == "🌫 Air Quality" else "none"
    temp_check_box.layout.display = "flex" if page == "🌤 Weather Overview" else "none"
    wind_check_box.layout.display = "flex" if page == "🌤 Weather Overview" else "none"

    with output:
        clear_output(wait=True)

        # Select which graphs to render
        if page == "🌤 Weather Overview":
            figs = weather_graphs(df_sub, temp_checkbox.value, wind_checkbox.value)
        elif page == "🌫 Air Quality":
            figs = air_graphs(df_sub, list(pollutant_check.value))
        else:
            figs = corr_graphs(df_sub)

        # If no figures, show message
        if not figs:
            display(widgets.HTML("<p style='color:red;'>No data available.</p>"))
            return

        # Display all Plotly figures side by side
        outputs = []
        for fig in figs:
            out = widgets.Output()
            with out:
                display(fig)  # Plotly figures display directly in Jupyter
            outputs.append(out)

        # Arrange figures in a row
        display(
            widgets.HBox(
                outputs,
                layout=widgets.Layout(
                    justify_content="center",
                    gap="25px",
                    width="100%"
                )
            )
        )

In [11]:
# Update Dashboard Graph + Filters + Pages
def update(*_):
    # Filter data by selected year, month, and day
    df_sub = df[
        df["year"].isin(year_check.value) &
        df["month"].isin(month_check.value) &
        df["day_of_week"].isin(day_check.value)
    ]

    # If no data found, show message
    with output:
        clear_output(wait=True)
        if df_sub.empty:
            display(widgets.HTML("<p style='color:red;'>No data for this selection.</p>"))
            return

    # Render the selected page
    render_page(df_sub, nav.value)

for group in [year_check,month_check,day_check,pollutant_check,nav,temp_checkbox,wind_checkbox]:
    group.observe(update,names="value")

# Dashboard Layout
filter_row=widgets.HBox(
    [year_box,month_box,day_box,pollutant_box,temp_check_box,wind_check_box],
    layout=widgets.Layout(justify_content="center",gap="20px",padding="10px")
)
centered_nav_container = widgets.HBox(
    [nav],
    layout=widgets.Layout(justify_content='center')
)
dashboard=widgets.VBox([header,centered_nav_container,filter_row,output],
                       layout=widgets.Layout(width="100%",padding="5px"))
display(dashboard)
update()